In [81]:
import numpy as np
import midi



from miditoolkit.midi import parser as mid_parser  
from miditoolkit.midi import containers as ct
from miditoolkit.midi import parser as mid_parser
from miditoolkit.pianoroll import parser as pr_parser
from miditoolkit.pianoroll import utils
import os
import numpy as np
from sklearn.utils import shuffle

In [82]:
# import music21
# score = music21.converter.parse('beethoven_hammerklavier_1.mid')
# key = score.analyze('key')
# print(key.tonic.name, key.mode)


Formatting Data Input

In [83]:
import random
train_samples = [] # array of array frequencies
train_labels = []  # [ [1, 0, 0,0... ], [ 0,0,0,1,0,0,0....] , ...  ]
keys = ["CfMaj", "GfMaj", "DfMaj", "AfMaj", "EfMaj", "BfMaj", "FMaj", "CMaj", "GMaj", "DMaj", "AMaj", "EMaj", "BMaj", "FsMaj",  "CsMaj",
"AfMin", "EfMin", "BfMin", "FMin", "CMin", "GMin", "DMin", "AMin", "EMin", "BMin", "FsMin", "CsMin", "GsMin", "DsMin", "AsMin"]


totalNotes = 0
for filename in os.listdir():
    if filename.endswith(".mid"): 
        # print("hello")
        mido_obj = mid_parser.MidiFile(filename)
        for j in range(len(mido_obj.instruments)):
            notes = mido_obj.instruments[j].notes
            totalNotes += 1
            random.shuffle(notes)
            # print("length of notes: ", len(notes))
            LABEL = []
            for i in range(len(keys)):
                try:
                    filename.index(keys[i])
                except ValueError:
                    continue
                else:
                    temp = [0]*30
                    temp[i] = 1
                    
                    # print(temp)
                    # train_labels = np.append(train_labels, temp, axis = 0)

                    LABEL = temp
                    break

            frequency = [0]*12 
            
            for i in range(len(notes)):
                if i % 100 == 0 and i != 0:
                    
                    total = sum(frequency)
                    
                    for x in range(len(frequency)):
                        frequency[x] = frequency[x] / total

                    train_samples.append(frequency)
                    train_labels.append(LABEL)

                    frequency = [0] * 12
                frequency[notes[i].pitch % 12] += 1
        
        
        
        

train_samples = np.array(train_samples)
train_labels = np.array(train_labels)
train_labels, train_samples = shuffle(train_labels, train_samples)

# print(train_samples[100])           
# print(train_labels[100])
print("# of inputs ", len(train_samples))

pitchToNotes = [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23 ] # goes from C to B, 12 semitones, any factors of number = note
# C, C#, D, D#, E, F, F#, G, G#, A, A#, B
print(totalNotes)
# print(train_samples)

# of inputs  1018
92


Model Imports

In [84]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy


Building the Model

In [85]:
model = Sequential([
    Dense(units = 24, input_shape = (12,), activation = 'relu'),
    
    Dense(units = 48, activation = 'relu'),
    
    
    Dense(units = 30, activation = "softmax"),
    
])
model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_52 (Dense)            (None, 24)                312       
                                                                 
 dense_53 (Dense)            (None, 48)                1200      
                                                                 
 dense_54 (Dense)            (None, 30)                1470      
                                                                 
Total params: 2,982
Trainable params: 2,982
Non-trainable params: 0
_________________________________________________________________


Setup internal Parameters

In [86]:
model.compile(optimizer = Adam(learning_rate = 0.005), loss = 'categorical_crossentropy', metrics = ['accuracy'])

Train + Validate

In [87]:
model.fit(x = train_samples, y = train_labels,  validation_split = 0.2, batch_size = 32, epochs = 60, shuffle = True, verbose = 2)

Epoch 1/60
26/26 - 1s - loss: 2.9552 - accuracy: 0.1916 - val_loss: 2.4604 - val_accuracy: 0.3529 - 504ms/epoch - 19ms/step
Epoch 2/60
26/26 - 0s - loss: 2.2286 - accuracy: 0.3010 - val_loss: 1.9198 - val_accuracy: 0.3431 - 52ms/epoch - 2ms/step
Epoch 3/60
26/26 - 0s - loss: 1.6363 - accuracy: 0.4521 - val_loss: 1.3997 - val_accuracy: 0.6324 - 54ms/epoch - 2ms/step
Epoch 4/60
26/26 - 0s - loss: 1.2181 - accuracy: 0.5971 - val_loss: 1.0786 - val_accuracy: 0.6029 - 50ms/epoch - 2ms/step
Epoch 5/60
26/26 - 0s - loss: 0.9848 - accuracy: 0.6560 - val_loss: 0.9065 - val_accuracy: 0.7010 - 52ms/epoch - 2ms/step
Epoch 6/60
26/26 - 0s - loss: 0.8293 - accuracy: 0.7420 - val_loss: 0.7905 - val_accuracy: 0.7010 - 54ms/epoch - 2ms/step
Epoch 7/60
26/26 - 0s - loss: 0.6827 - accuracy: 0.7924 - val_loss: 0.6473 - val_accuracy: 0.7990 - 53ms/epoch - 2ms/step
Epoch 8/60
26/26 - 0s - loss: 0.5849 - accuracy: 0.8366 - val_loss: 0.5282 - val_accuracy: 0.8873 - 54ms/epoch - 2ms/step
Epoch 9/60
26/26 - 0s 